In [23]:
from PIL import Image
import numpy as np
import os

# Add the OpenSlide DLL directory
openslide_path = r'C:\Users\Valentina\openslide-win64-20220811\bin'
if hasattr(os, 'add_dll_directory'):
    # Python 3.8+
    with os.add_dll_directory(openslide_path):
        from openslide import OpenSlide
else:
    # Earlier Python versions
    if openslide_path not in os.environ['PATH']:
        os.environ['PATH'] = openslide_path + os.pathsep + os.environ['PATH']
    from openslide import OpenSlide


In [24]:
path_wsi = r'\\10.99.68.52\Kiemendata\Valentina Matos\LG HG PanIN project\LGHG segmentation'
tenx_path = os.path.join(path_wsi, '10x_python')
fivex_path = os.path.join(path_wsi,'5x_python')
onex_path = os.path.join(path_wsi, '1x_python')
for path in [tenx_path, fivex_path, onex_path]:
    if not os.path.exists(path):
        os.mkdir(path)

In [25]:
wsis = [_ for _ in os.listdir(path_wsi) if _.endswith('.ndpi') or _.endswith('.svs')]
wsis2 = [os.path.splitext(_)[0] for _ in wsis]
tenx = [_ for _ in os.listdir(tenx_path) if _.endswith('.tif')]
tenx2 = [os.path.splitext(_)[0] for _ in tenx]
missingimages = [_ for _ in wsis2 if _ not in tenx2]

In [26]:
def process_image(missingimage, path_wsi, output_path, rsf):
    print(f"Processing: {missingimage}")
    file_extension = next(ext for ext in ['.ndpi', '.svs'] if os.path.exists(os.path.join(path_wsi, missingimage + ext)))
    wsi = OpenSlide(os.path.join(path_wsi, missingimage + file_extension))
    svs_img = wsi.read_region(location=(0,0), level=0, size=wsi.level_dimensions[0]).convert('RGB')
    
    if rsf == 1:  # 10x
        resize_factorx = rsf / float(wsi.properties['openslide.mpp-x'])
        resize_factory = rsf / float(wsi.properties['openslide.mpp-y'])
    elif rsf == 4:  # 5x
        resize_factorx = 1 / rsf
        resize_factory = 1 / rsf
    else:  # 1x
        resize_factorx = rsf / float(wsi.properties['openslide.mpp-x'])
        resize_factory = rsf / float(wsi.properties['openslide.mpp-y'])
    
    resize_dimension = tuple([int(np.ceil(wsi.dimensions[0]/resize_factorx)), int(np.ceil(wsi.dimensions[1]/resize_factory))])
    svs_img = svs_img.resize(resize_dimension, resample=Image.NEAREST)
    svs_img.save(os.path.join(output_path, missingimage + '.tif'), resolution=1, resolution_unit=1, quality=100, compression=None)


In [ ]:
# Process images for 10x, 5x, and 1x
# for magnification, (output_path, rsf) in [('10x', (tenx_path, 1)), ('5x', (fivex_path, 4)), ('1x', (onex_path, 8))]:
for magnification, (output_path, rsf) in [('5x', (fivex_path, 4))]:
    print(f"Creating {magnification} tif images")
    for idx, missingimage in enumerate(missingimages):
        print(f"{idx}/{len(missingimages)}")
        process_image(missingimage, path_wsi, output_path, rsf)
print("Processing complete!")

Creating 5x tif images
0/72
Processing: AB_04_0001


In [ ]:
# Create 10x tif images
for idx,missingimage in enumerate(missingimages):
    print(idx,'/',len(missingimages),' processing: ',missingimage)
    wsi = OpenSlide(os.path.join(path_wsi,missingimage+'.ndpi'))
    rsf = 1
    svs_img = wsi.read_region(location=(0,0),level=0,size=wsi.level_dimensions[0]).convert('RGB')
    resize_factorx = rsf/float(wsi.properties['openslide.mpp-x']) #8um = 1.25x #4um = 2.5x, #2um=5x, 1um=10x, 0.5um=20x, 0.25um=40x
    resize_factory = rsf/float(wsi.properties['openslide.mpp-y'])
    resize_dimension = tuple([int(np.ceil(wsi.dimensions[0]/resize_factorx)),int(np.ceil(wsi.dimensions[1]/resize_factory))])
    svs_img = svs_img.resize(resize_dimension,resample=Image.NEAREST)
    svs_img.save(os.path.join(tenx_path,missingimage+'.tif'),resolution=1,resolution_unit=1,quality=100,compression=None)

0 / 108  processing:  PDAC-VI-KH-002_0103


ValueError: image has wrong mode

In [11]:
# Create 5x tif images
for idx,missingimage in enumerate(missingimages):
    print(idx,'/',len(missingimages),' processing: ',missingimage)
    wsi = OpenSlide(os.path.join(path_wsi,missingimage+'.ndpi'))
    svs_img = wsi.read_region(location=(0,0),level=0,size=wsi.level_dimensions[0]).convert('RGB')
    rsf = 4
    resize_factorx = 1/rsf #8um = 1.25x #4um = 2.5x, #2um=5x, 1um=10x, 0.5um=20x, 0.25um=40x
    resize_factory = 1/rsf
    resize_dimension = tuple([int(np.ceil(wsi.dimensions[0]/resize_factorx)),int(np.ceil(wsi.dimensions[1]/resize_factory))])
    svs_img = svs_img.resize(resize_dimension,resample=Image.NEAREST)
    svs_img.save(os.path.join(tenx_path,missingimage+'.tif'),resolution=1,resolution_unit=1,quality=100,compression=None)

0 / 66  processing:  AB_04_0001


KeyboardInterrupt: 

In [ ]:
# Create 1x tif images
for idx,missingimage in enumerate(missingimages):
    print(idx,'/',len(missingimages),' processing: ',missingimage)
    wsi = OpenSlide(os.path.join(path_wsi,missingimage+'.ndpi'))
    svs_img = wsi.read_region(location=(0,0),level=0,size=wsi.level_dimensions[0]).convert('RGB')
    rsf = 8
    resize_factorx = rsf/float(wsi.properties['openslide.mpp-x']) #8um = 1.25x #4um = 2.5x, #2um=5x, 1um=10x, 0.5um=20x, 0.25um=40x
    resize_factory = rsf/float(wsi.properties['openslide.mpp-y'])
    resize_dimension = tuple([int(np.ceil(wsi.dimensions[0]/resize_factorx)),int(np.ceil(wsi.dimensions[1]/resize_factory))])
    svs_img2_5x = svs_img.resize(resize_dimension,resample=Image.NEAREST)
    svs_img2_5x.save(os.path.join(onex_path,missingimage+'.tif'),resolution=2_5,resolution_unit=1,quality=100,compression=None)
    svs_img.save(os.path.join(tenx_path,missingimage+'.tif'),resolution=1,resolution_unit=1,quality=100,compression=None)